In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import torch

print(torch.cuda.is_available())

True


In [3]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.__config__.show())

2.3.1+cu121
12.1
PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DN

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from steering_vectors.utils import DeviceManager

model_size = "7b"
model_name = f"meta-llama/Llama-2-{model_size}-chat-hf"

DeviceManager.instance().set_device("cuda")

bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    load_in_4bit=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


def get_model_and_tokenizer(model_name: str):
    device = DeviceManager.instance().get_device()
    tokenizer = AutoTokenizer.from_pretrained(model_name, device=device)
    # Note: you must have installed 'accelerate', 'bitsandbytes' to load in 8bit
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
    return model, tokenizer


model, tokenizer = get_model_and_tokenizer(model_name)
print(model)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [5]:
# Download the CAA dataset
!wget https://raw.githubusercontent.com/nrimsky/CAA/main/datasets/generate/sycophancy/generate_dataset.json
!wget https://raw.githubusercontent.com/nrimsky/CAA/main/datasets/test/sycophancy/test_dataset_ab.json

--2024-07-04 15:26:43--  https://raw.githubusercontent.com/nrimsky/CAA/main/datasets/generate/sycophancy/generate_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871127 (851K) [text/plain]
Saving to: ‘generate_dataset.json.3’

generate_dataset.js 100%[===================>] 850.71K  --.-KB/s    in 0.1s    

2024-07-04 15:26:43 (8.13 MB/s) - ‘generate_dataset.json.3’ saved [871127/871127]

--2024-07-04 15:26:44--  https://raw.githubusercontent.com/nrimsky/CAA/main/datasets/test/sycophancy/test_dataset_ab.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent

In [6]:
import json
import random

# Define a shorthand type for model-written eval datum
MWEData = list[dict[str, str]]

train_data: list[MWEData] = json.load(open("generate_dataset.json", "r"))
test_data: list[MWEData] = json.load(open("test_dataset_ab.json", "r"))

random.seed(42)
random.shuffle(train_data)
random.shuffle(test_data)
train_data = train_data
test_data = test_data

In [7]:
B_INST = "[INST]"
E_INST = "[/INST]"
SYS_PROMPT = "You are a helpful, honest assistant."
B_SYS = "<<SYS>>"
E_SYS = "<</SYS>>"
BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
# Note: No EOS token is used in original codebase
prompt_template = f"{BOS_TOKEN} {B_INST} \n{B_SYS}{SYS_PROMPT}{E_SYS}\n\n {{question}} {E_INST} {{answer}}"


def make_pos_neg_pair(mwe_data: MWEData) -> tuple[str, str]:
    """Creates a (positive, negative) pair for getting contrastive activations"""
    pos = prompt_template.format(
        question=mwe_data["question"], answer=mwe_data["answer_matching_behavior"]
    )
    neg = prompt_template.format(
        question=mwe_data["question"], answer=mwe_data["answer_not_matching_behavior"]
    )
    return pos, neg


def make_dataset(list_mwe_data: list[MWEData]) -> list[tuple[str, str]]:
    """Creates a list of (positive, negative) pairs for getting contrastive activations"""
    return [make_pos_neg_pair(mwe_data) for mwe_data in list_mwe_data]

In [8]:
train_dataset = make_dataset(train_data)
test_dataset = make_dataset(test_data)

In [9]:
from steering_vectors import train_steering_vector, SteeringVector

steering_vector: SteeringVector = train_steering_vector(
    model,
    tokenizer,
    train_dataset[:10],
    move_to_cpu=True,
    # NOTE: You can specify a list[int] of desired layer indices
    # If layers is None, then all layers are used
    # Here, layer 15 is the layer where sycophancy steering worked best in the CAA paper
    # for both Llama-2-7b-chat and Llama-2-13b-chat.
    layers=[15],
    # NOTE: The second last token corresponds to the A/B position
    # which is where we believe the model makes its decision
    read_token_index=-2,
    show_progress=True,
)

Training steering vector: 100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


In [10]:
for multiplier in (-1, 0, 1):
    with steering_vector.apply(model, multiplier=multiplier, min_token_index=0) as handle:
        print(handle)

SteeringPatchHandle(model_hooks=[<torch.utils.hooks.RemovableHandle object at 0x7ab3a6ba63e0>], module_names=['model.layers.15.output'])
SteeringPatchHandle(model_hooks=[<torch.utils.hooks.RemovableHandle object at 0x7ab3a6ba68c0>], module_names=['model.layers.15.output'])
SteeringPatchHandle(model_hooks=[<torch.utils.hooks.RemovableHandle object at 0x7ab3a6ba63e0>], module_names=['model.layers.15.output'])
